# Manipulate a function
Tutorial by Jonas Wilfert, Tobias Thummerer

## License
Copyright (c) 2021 Tobias Thummerer, Lars Mikelsons, Josef Kircher, Johannes Stoljar, Jonas Wilfert

Licensed under the MIT license. See [LICENSE](https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

## Motivation
This Julia Package *FMI.jl* is motivated by the use of simulation models in Julia. Here the FMI specification is implemented. FMI (*Functional Mock-up Interface*) is a free standard ([fmi-standard.org](http://fmi-standard.org/)) that defines a container and an interface to exchange dynamic models using a combination of XML files, binaries and C code zipped into a single file. The user can thus use simulation models in the form of an FMU (*Functional Mock-up Units*). Besides loading the FMU, the user can also set values for parameters and states and simulate the FMU both as co-simulation and model exchange simulation.

## Introduction to the example
This example shows how to parallelize the computation of an FMU in FMI.jl. We can compute a batch of FMU-evaluations in parallel with different initial settings.
Parallelization can be achieved using multithreading or using multiprocessing. This example shows **multithreading**, check `multiprocessing.ipynb` for multiprocessing.
Advantage of multithreading is a lower communication overhead as well as lower RAM usage.
However in some cases multiprocessing can be faster as the garbage collector is not shared.


The model used is a one-dimensional spring pendulum with friction. The object-orientated structure of the *SpringFrictionPendulum1D* can be seen in the following graphic.

![svg](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/pics/SpringFrictionPendulum1D.svg?raw=true)  


## Target group
The example is primarily intended for users who work in the field of simulations. The example wants to show how simple it is to use FMUs in Julia.


## Other formats
Besides, this [Jupyter Notebook](https://github.com/thummeto/FMI.jl/blob/main/example/parallel.ipynb) there is also a [Julia file](https://github.com/thummeto/FMI.jl/blob/main/example/parallel.jl) with the same name, which contains only the code cells and for the documentation there is a [Markdown file](https://github.com/thummeto/FMI.jl/blob/main/docs/src/examples/parallel.md) corresponding to the notebook.  


## Getting started

### Installation prerequisites
|     | Description                       | Command                   | Alternative                                    |   
|:----|:----------------------------------|:--------------------------|:-----------------------------------------------|
| 1.  | Enter Package Manager via         | ]                         |                                                |
| 2.  | Install FMI via                   | add FMI                   | add " https://github.com/ThummeTo/FMI.jl "     |
| 3.  | Install FMIZoo via                | add FMIZoo                | add " https://github.com/ThummeTo/FMIZoo.jl "  |
| 4.  | Install FMICore via               | add FMICore               | add " https://github.com/ThummeTo/FMICore.jl " |
| 5.  | Install Folds via                 | add Folds                 |                                                |
| 6.  | Install BenchmarkTools via        | add BenchmarkTools        |                                                |

## Code section

To run the example, the previously installed packages must be included. 

In [1]:
# imports
using FMI
using FMIZoo
using Folds
using BenchmarkTools

┌ Warning: Package SuiteSparse_jll does not have libblastrampoline_jll in its dependencies:
│ - If you have SuiteSparse_jll checked out for development and have
│   added libblastrampoline_jll as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with SuiteSparse_jll
└ Loading libblastrampoline_jll into SuiteSparse_jll from project dependency, future warnings for SuiteSparse_jll are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Sundials_jll [fb77eaff-e24c-56d4-86b1-d163f2edb164]
│   exception = Required dependency SuiteSparse_jll [bea87d4a-7f5b-5778-9afe-8cc45184846c] failed to load from a cache file.
└ @ Base loading.jl:1132
┌ Warning: Package SuiteSparse_jll does not have libblastrampoline_jll in its dependencies:
│ - If you have SuiteSparse_jll checked out for development and have
│   added libblastrampoline_jll as a dependency but hav

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Sundials_jll [fb77eaff-e24c-56d4-86b1-d163f2edb164]
│   exception = Required dependency SuiteSparse_jll [bea87d4a-7f5b-5778-9afe-8cc45184846c] failed to load from a cache file.
└ @ Base loading.jl:1132
┌ Warning: Package SuiteSparse_jll does not have libblastrampoline_jll in its dependencies:
│ - If you have SuiteSparse_jll checked out for development and have
│   added libblastrampoline_jll as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with SuiteSparse_jll
└ Loading libblastrampoline_jll into SuiteSparse_jll from project dependency, future warnings for SuiteSparse_jll are suppressed.
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
│   exception = ErrorException("Required dependency SuiteSparse_jl

First, check the amount of available threads:

In [2]:
Threads.nthreads()

4

If the number of available threads doesn't match your expections, you can increase the number of threads available to the Julia process like described [here](https://docs.julialang.org/en/v1/manual/multi-threading/#Starting-Julia-with-multiple-threads).

### Simulation setup

Next, the start time and end time of the simulation are set. Here we also decide the size of the batch.

In [3]:
t_start = 0.0
t_step = 0.1
t_stop = 10.0
tspan = (t_start, t_stop)
tData = collect(t_start:t_step:t_stop)

# Best if batchSize is a multiple of the threads/cores
batchSize = Threads.nthreads()

# Define an array of arrays randomly
input_values = collect(collect.(eachrow(rand(batchSize,2))))


4-element Vector{Vector{Float64}}:
 [0.9477237606768271, 0.08554214067501187]
 [0.9505517343695893, 0.36607879620662687]
 [0.8764693360290572, 0.2622322961937148]
 [0.7278996310219334, 0.01357781770958899]

We need to instantiate one FMU for each parallel execution, as they cannot be easily shared among different threads.

In [7]:
# a single FMU to compare the performance
realFMU = fmiLoad("SpringPendulum1D", "Dymola", "2022x")

# the FMU batch
realFMUBatch = [fmiLoad("SpringPendulum1D", "Dymola", "2022x") for _ in 1:batchSize]

┌ Info: fmi2Unzip(...): Successfully unzipped 153 files at `C:\Users\thummeto\AppData\Local\Temp\fmijl_816FfN\SpringPendulum1D`.
└ @ FMIImport C:\Users\thummeto\Documents\FMIImport.jl\src\FMI2_ext.jl:76
┌ Info: fmi2Load(...): FMU resources location is `file:///C:/Users/thummeto/AppData/Local/Temp/fmijl_816FfN/SpringPendulum1D/resources`
└ @ FMIImport C:\Users\thummeto\Documents\FMIImport.jl\src\FMI2_ext.jl:192
┌ Info: fmi2Load(...): FMU supports both CS and ME, using CS as default if nothing specified.
└ @ FMIImport C:\Users\thummeto\Documents\FMIImport.jl\src\FMI2_ext.jl:195
┌ Info: fmi2Unzip(...): Successfully unzipped 153 files at `C:\Users\thummeto\AppData\Local\Temp\fmijl_bWV1QI\SpringPendulum1D`.
└ @ FMIImport C:\Users\thummeto\Documents\FMIImport.jl\src\FMI2_ext.jl:76
┌ Info: fmi2Load(...): FMU resources location is `file:///C:/Users/thummeto/AppData/Local/Temp/fmijl_bWV1QI/SpringPendulum1D/resources`
└ @ FMIImport C:\Users\thummeto\Documents\FMIImport.jl\src\FMI2_ext.jl:192
┌ I

4-element Vector{FMU2}:
 Model name:        SpringPendulum1D
Type:              1
 Model name:        SpringPendulum1D
Type:              1
 Model name:        SpringPendulum1D
Type:              1
 Model name:        SpringPendulum1D
Type:              1

We define a helper function to calculate the FMU solution and combine it into an Matrix.

In [5]:
function runCalcFormatted(fmu::FMU2, x0::Vector{Float64}, recordValues::Vector{String}=["mass.s", "mass.v"])
    data = fmiSimulateME(fmu, t_start, t_stop; recordValues=recordValues, saveat=tData, x0=x0, showProgress=false, dtmax=1e-4)
    return reduce(hcat, data.states.u)
end

runCalcFormatted (generic function with 2 methods)

Running a single evaluation is pretty quick, therefore the speed can be better tested with BenchmarkTools.

In [8]:
@benchmark data = runCalcFormatted(realFMU, rand(2))

BenchmarkTools.Trial: 16 samples with 1 evaluation.
 Range (min … max):  306.955 ms … 359.793 ms  ┊ GC (min … max): 2.79% … 4.64%
 Time  (median):     315.014 ms               ┊ GC (median):    5.18%
 Time  (mean ± σ):   318.562 ms ±  13.562 ms  ┊ GC (mean ± σ):  4.60% ± 1.16%

  █    ▁▁█▁█▁██                            ▁                  ▁  
  █▁▁▁▁████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  307 ms           Histogram: frequency by time          360 ms <

 Memory estimate: 119.32 MiB, allocs estimate: 3002338.

### Single Threaded Batch Execution
To compute a batch we can collect multiple evaluations. In a single threaded context we can use the same FMU for every call.

In [11]:
println("Single Threaded")
@benchmark collect(runCalcFormatted(realFMU, i) for i in input_values)

Single Threaded


BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.251 s …   1.329 s  ┊ GC (min … max): 4.09% … 5.10%
 Time  (median):     1.290 s              ┊ GC (median):    4.53%
 Time  (mean ± σ):   1.290 s ± 34.650 ms  ┊ GC (mean ± σ):  4.57% ± 0.43%

  █               █                        █              █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.25 s         Histogram: frequency by time        1.33 s <

 Memory estimate: 477.29 MiB, allocs estimate: 12009352.

### Multithreaded Batch Execution
In a multithreaded context we have to provide each thread it's own fmu, as they are not thread safe.
To spread the execution of a function to multiple threads, the library `Folds` can be used.

In [12]:
println("Multi Threaded")
@benchmark Folds.collect(runCalcFormatted(fmu, i) for (fmu, i) in zip(realFMUBatch, input_values))

Multi Threaded


BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  572.746 ms … 646.224 ms  ┊ GC (min … max):  5.75% … 11.94%
 Time  (median):     602.312 ms               ┊ GC (median):    11.38%
 Time  (mean ± σ):   608.197 ms ±  23.474 ms  ┊ GC (mean ± σ):  10.20% ±  2.59%

  █         █        █  █ █        █         █     █          █  
  █▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁█▁█▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█ ▁
  573 ms           Histogram: frequency by time          646 ms <

 Memory estimate: 477.29 MiB, allocs estimate: 12009395.

As you can see, there is a significant speed-up in the median execution time. But: The speed-up is often much smaller than `Threads.nthreads()`, this has different reasons. For a rule of thumb, the speed-up should be around `n/2` on a `n`-core-processor with `n` threads for the Julia process.

### Unload FMU

After calculating the data, the FMU is unloaded and all unpacked data on disc is removed.

In [14]:
fmiUnload(realFMU)
fmiUnload.(realFMUBatch)

16-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

### Summary

In this tutorial it is shown how multi threading with `Folds.jl` can be used to improve the performance for calculating a Batch of FMUs.